In [1]:
using CSV, Tables, LinearAlgebra, Random, Gurobi, JuMP, DataFrames

In [2]:
df = DataFrame(CSV.File("final_data.csv"));

In [63]:
names(df)

38-element Vector{String}:
 "Datetime"
 "Day"
 "Month"
 "Hour"
 "Year"
 "Dual Fuel_Gen_MW"
 "Hydro_Gen_MW"
 "Natural Gas_Gen_MW"
 "Nuclear_Gen_MW"
 "Other Fossil Fuels_Gen_MW"
 "Solar_Gen_MW"
 "Wind_Gen_MW"
 "temp"
 ⋮
 "uvindex"
 "conditions_Clear"
 "conditions_Overcast"
 "conditions_Partly_cloudy"
 "conditions_Rain_overcast"
 "conditions_Rain_partly_cloudy"
 "conditions_Snow_overcast"
 "conditions_Snow_rain_overcast"
 "preciptype_no_precipitation"
 "preciptype_rain"
 "preciptype_rain_and_snow"
 "preciptype_snow"

In [3]:
describe(df)

,variable,mean,min,median,max
,Symbol,Union…,Any,Union…,Any
1,Datetime,,2021-01-01 00:00:00,,2021-12-31 23:00:00
2,Day,15.7207,1,16.0,31
3,Month,6.52643,1,7.0,12
4,Hour,11.5011,0,12.0,23
5,Year,2021.0,2021,2021.0,2021
6,Dual Fuel_Gen_MW,45271.4,11226.0,41720.0,170491.0
7,Hydro_Gen_MW,38502.6,13771.0,39276.0,72251.0
8,Natural Gas_Gen_MW,38654.3,12595.0,37104.0,100129.0
9,Nuclear_Gen_MW,43465.0,28202.0,40014.0,89400.0


In [4]:
data = CSV.File("final_data.csv", header=true) |> Tables.matrix;

In [19]:
#Part d
function stable_regression(X, y, k)
    m = Model(Gurobi.Optimizer)
    set_optimizer_attribute(m, "OutputFlag", 0)
    
    n, p = size(X)
    
    @variable(m, B[1:p])
    @variable(m, u[1:n])
    @variable(m, θ)
    #println(typeof(B))
    #println(typeof(u))
    #println(typeof(y))
    #println(typeof(X))
    @constraint(m, u .+ θ .>= (y - X*B).^2)
    @constraint(m, u .>= 0)
    @objective(m, Min, k*θ + sum(u))

    optimize!(m)
    return value.(B)
end;

In [6]:
y_solar = data[:,11];
X = data[:,13:38];
y_wind = data[:,12];


In [7]:
X

8759×26 Matrix{Any}:
 32.4  29.7  24.0  70.72  0.0  0  0.0   …  0  0  0  1  0  0  0  0  1  0  0  0
 32.5  32.5  24.1  70.76  0.0  0  0.0      0  0  1  0  0  0  0  0  1  0  0  0
 31.6  28.3  24.2  73.84  0.0  0  0.0      0  0  1  0  0  0  0  0  1  0  0  0
 31.6  26.8  25.1  76.66  0.0  0  0.0      0  0  1  0  0  0  0  0  1  0  0  0
 31.8  27.2  25.2  76.31  0.0  0  0.0      0  0  1  0  0  0  0  0  1  0  0  0
 29.5  23.6  24.1  79.99  0.0  0  0.0   …  0  0  0  1  0  0  0  0  1  0  0  0
 29.7  23.9  23.2  76.63  0.0  0  0.0      0  0  0  1  0  0  0  0  1  0  0  0
 28.9  22.9  21.3  72.8   0.0  0  0.0      0  0  0  1  0  0  0  0  1  0  0  0
 28.1  24.7  21.0  74.55  0.0  0  0.0      0  0  0  1  0  0  0  0  1  0  0  0
 28.8  22.7  21.2  72.71  0.0  0  2.35     1  0  0  1  0  0  0  0  1  0  0  0
 30.7  23.1  22.1  69.99  0.0  0  0.0   …  1  0  0  1  0  0  0  0  1  0  0  0
 31.9  24.0  22.1  66.79  0.0  0  0.0      1  0  1  0  0  0  0  0  1  0  0  0
 33.0  23.8  21.3  61.67  0.0  0  0.0      

In [8]:
function MSE(y, pred) 
    return sum((y-pred).^2)/length(y)
end;

### SOLAR: BEST SPLIT
80% TRAIN 20% VALIDATION

In [64]:
n,p = size(X)
MSE_best, k_best = Inf, Inf
X_train_best, X_valid_best, y_train_best, y_valid_best = nothing, nothing, nothing, nothing
for k in [4380,4818,5255,5693,6131,6569,7007,7445,7883] #we look for the best split of the data
    X_train = Matrix(X[1:k,:])
    y_train = y_solar[1:k]
    X_valid = Matrix(X[k:n,:])
    y_valid = y_solar[k:n]

    β_temp = stable_regression(X_train, y_train, k)
    MSE_temp = MSE(y_valid, X_valid*β_temp)
    if MSE_temp < MSE_best
        MSE_best = MSE_temp
        k_best = k 
        X_train_best = X_train
        X_valid_best = X_valid
        y_train_best = y_train
        y_valid_best = y_valid
    end
end


Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19


In [65]:
k_best #best split is 80% training and 20% validation

7007

In [66]:
MSE_best

87527.62999038263

In [68]:
X_train_solar = X_train_best
X_valid_solar = X_valid_best
y_train_solar = y_train_best
y_valid_solar = y_valid_best;

In [70]:
#transform back to dataframe to understand the coefficients
X_train_solar_df=DataFrame(X_train_solar, :auto)
X_valid_solar_df=DataFrame(X_valid_solar, :auto)
#rename columns with list of values
rename!(X_train_solar_df, names(df[:, 13:38]))
rename!(X_valid_solar_df, names(df[:, 13:38]))

#transform vector into dataframe 
y_train_solar=DataFrame(Solar_Gen_MW = y_train_solar)
y_valid_solar=DataFrame(Solar_Gen_MW = y_valid_solar)
#save as CSV
CSV.write("X_train_solar.csv", X_train_solar_df)
CSV.write("X_valid_solar.csv", X_valid_solar_df)
CSV.write("y_train_solar.csv", y_train_solar)
CSV.write("y_valid_solar.csv", y_valid_solar);

In [18]:
stable_regression(X_train_solar, y_train_solar, k_best)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Vector{VariableRef}
Vector{VariableRef}
Vector{Any}
Matrix{Any}


26-element Vector{Float64}:
     4.207740994493473
    -1.5304394082907644
    -0.8818621284265042
     1.911293646267334
   216.50760347355262
   -67.78477706341644
     7.676416113646777
    10.275899094131088
     0.585991507048097
     0.11909803430672675
     3.824210734559088
     0.10440191916407941
    -3.2841605178075204
     0.42272270437481924
     5.48544289536271
  7855.683015826097
  7899.393104172791
  7894.086678773774
 -7421.082271086897
 -7354.269744753053
  5049.81165926949
  5473.127928959473
 -9119.232502132412
 12937.610550544317
    56.55979982185331
   458.14076390341813

### WIND: BEST SPLIT 
90% TRAIN 10% VALIDATION

In [71]:
n,p = size(X)
MSE_best, k_best = Inf, Inf
X_train_best, X_valid_best, y_train_best, y_valid_best = nothing, nothing, nothing, nothing
for k in [4380,4818,5255,5693,6131,6569,7007,7445,7883] #we look for the best split of the data
    X_train = Matrix(X[1:k,:])
    y_train = y_wind[1:k]
    X_valid = Matrix(X[k:n,:])
    y_valid = y_wind[k:n]

    β_temp = stable_regression(X_train, y_train, k)
    MSE_temp = MSE(y_valid, X_valid*β_temp)
    if MSE_temp < MSE_best
        MSE_best = MSE_temp
        k_best = k 
        X_train_best = X_train
        X_valid_best = X_valid
        y_train_best = y_train
        y_valid_best = y_valid
    end
end

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19


In [72]:
println(k_best) #best split is 90% training and 10% validation
println(MSE_best)

7883
1.1706974107038202e7


In [73]:
X_train_wind = X_train_best
y_train_wind = y_train_best
X_valid_wind = X_valid_best
y_valid_wind = y_valid_best;

In [74]:
#transform back to dataframe to understand the coefficients
X_train_wind_df=DataFrame(X_train_wind, :auto)
X_valid_wind_df=DataFrame(X_valid_wind, :auto)
#rename columns with list of values
rename!(X_train_wind_df, names(df[:, 13:38]))
rename!(X_valid_wind_df, names(df[:, 13:38]))

#transform vector into dataframe 
y_train_wind=DataFrame(Wind_Gen_MW = y_train_wind)
y_valid_wind=DataFrame(Wind_Gen_MW = y_valid_wind)
#save as CSV
CSV.write("X_train_wind.csv", X_train_wind_df)
CSV.write("y_train_wind.csv", y_train_wind)
CSV.write("X_valid_wind.csv", X_valid_wind_df)
CSV.write("y_valid_wind.csv", y_valid_wind);